In [ ]:
from google.colab import files
import pandas as pd

# Upload file CSV yang berisi link berita
uploaded = files.upload()

# Asumsikan nama file adalah 'detik_bpjs_articles.csv'
df_links = pd.read_csv('detik_bpjs_word_only_articles.csv', sep=';')

# Tampilkan beberapa baris pertama untuk verifikasi
df_links.head()


Saving detik_bpjs_word_only_articles.csv to detik_bpjs_word_only_articles.csv


,title,link,category,date
0,"Buka-bukaan BPJS Kesehatan, Banyak RS 'Nakal' ...",https://health.detik.com/berita-detikhealth/d-...,detikHealth,"Jumat, 20 Sep 2024 06:00 WIB"
1,Temuan Praktik 'Nakal' RS ke BPJS Kesehatan: P...,https://health.detik.com/berita-detikhealth/d-...,detikHealth,"Kamis, 19 Sep 2024 14:02 WIB"
2,"BPJS Kesehatan Ungkap 3 RS Manipulasi Klaim, N...",https://health.detik.com/berita-detikhealth/d-...,detikHealth,"Kamis, 19 Sep 2024 12:28 WIB"
3,Berantas Rumah Sakit 'Nakal' Pelaku Klaim BPJS...,https://health.detik.com/detikhealth/d-7466486...,detikHealth,"Rabu, 31 Jul 2024 14:57 WIB"
4,Persi Kutuk Keras RS yang Curangi Klaim BPJS,https://health.detik.com/detiktv/d-7467310/per...,detikHealth,"Rabu, 31 Jul 2024 21:12 WIB"


In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

# Step 1: Function to extract text from each article
def extract_article_details(link):
    all_text = ""
    all_tags = []

    page_number = 1
    while True:
        # Update the link to the current page
        current_link = f"{link}/{page_number}"
        response = requests.get(current_link)

        if response.status_code != 200:
            print(f"Failed to retrieve article: {current_link}")
            break

        soup = BeautifulSoup(response.text, "html.parser")

        # Remove unwanted content
        for unwanted in soup.find_all(['div', 'span'], class_=['lihatjg', 'aevp_title','aevp']):
            unwanted.decompose()

        # Title
        title_tag = soup.find('h1', class_='detail__title')
        title = title_tag.get_text(strip=True) if title_tag else "No title"

        # Tags (collecting tags before removing them from the text)
        tag_container = soup.find('div', class_='detail__body-tag')
        if tag_container:
            tag_links = tag_container.find_all('a', class_='nav__item')
            tags = [tag.get_text(strip=True) for tag in tag_links]
            all_tags.extend(tags)  # Collect tags across pages

            # Remove tags from the body text
            for tag in tag_links:
                tag.decompose()

        # Article text
        text_tag = soup.find('div', class_='detail__body-text')
        text = text_tag.get_text(strip=True) if text_tag else "No text"
        all_text += text + "\n"  # Add text from this page

        # Date
        date_tag = soup.find('div', class_='detail__date')
        date = date_tag.get_text(strip=True) if date_tag else "No date"

        # Check for next page
        next_page_tag = soup.find('a', class_='detail__anchor-numb', attrs={'dtr-act': 'button selanjutnya'})
        if not next_page_tag:
            break  # No more pages, exit the loop

        page_number += 1  # Move to the next page

    tags_str = ', '.join(set(all_tags)) if all_tags else "No tags"  # Unique tags

    return {
        'title': title,
        'url': link,
        'text': all_text.strip(),
        'date': date,
        'tags': tags_str
    }

# Step 2: Loop to extract articles from all links in the CSV file
all_articles = []
for link in df_links['link']:
    print(f"Extracting article from {link}")

    article_details = extract_article_details(link)
    all_articles.append(article_details)

    # Delay to avoid sending requests too quickly
    time.sleep(2)

# Step 3: Convert results into a DataFrame
df_articles = pd.DataFrame(all_articles)

# Step 4: Export DataFrame to CSV with semicolon separator
df_articles.to_csv('detik_bpjs_article_details.csv', sep=';', index=False)

print("Data artikel berhasil diekspor ke 'detik_bpjs_article_details.csv'")


Extracting article from https://health.detik.com/berita-detikhealth/d-7548897/buka-bukaan-bpjs-kesehatan-banyak-rs-nakal-bikin-klaim-palsu-overbilling
Failed to retrieve article: https://health.detik.com/berita-detikhealth/d-7548897/buka-bukaan-bpjs-kesehatan-banyak-rs-nakal-bikin-klaim-palsu-overbilling/3
Extracting article from https://health.detik.com/berita-detikhealth/d-7547876/temuan-praktik-nakal-rs-ke-bpjs-kesehatan-pasien-bodong-sampai-overbilling
Extracting article from https://health.detik.com/berita-detikhealth/d-7547687/bpjs-kesehatan-ungkap-3-rs-manipulasi-klaim-negara-rugi-rp-34-miliar
Extracting article from https://health.detik.com/detikhealth/d-7466486/berantas-rumah-sakit-nakal-pelaku-klaim-bpjs-palsu
Extracting article from https://health.detik.com/detiktv/d-7467310/persi-kutuk-keras-rs-yang-curangi-klaim-bpjs
Extracting article from https://health.detik.com/berita-detikhealth/d-7428361/bpjs-kesehatan-launching-layanan-face-recognition-upayakan-keamanan-data
Extract